In [2]:
pip install psycopg2-binary


  Using cached psycopg2-binary-2.9.10.tar.gz (385 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for psycopg2-binary: filename=psycopg2_binary-2.9.10-cp312-cp312-macosx_10_13_universal2.whl size=246285 sha256=e05d1e8a737a1c3af6abc513401a3da8467cad5339245cb173b48dbd8973644b
  Stored in directory: /Users/gamaksahani/Library/Caches/pip/wheels/06/bc/a4/bad5bdabd4cf012a00e927db042e0e44d3a649596c548212be
Successfully built psycopg2-binary

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import re
from datetime import datetime

In [3]:
import psycopg2

def connect_postgres():
    conn = psycopg2.connect(
        dbname="metrics",
        user="postgres",
        password="sssawsds",
        host="localhost",   
        port="5432"        
    )
    return conn


In [4]:
from config import DB_CONFIG

try:
    conn = connect_postgres()
    print("Connection successful!")
    conn.close()
except Exception as e:
    print("Connection failed:", e)


✅ Connection successful!


In [10]:
def extract_metrics(text):
    patterns = {
        "Revenue from Operations": r"Revenue from Operations\s+([\d\.]+)\s*Cr",
        "EBITDA": r"EBITDA\s+([\d\.]+)\s*Cr",
        "PAT": r"PAT\s+([\d\.]+)\s*Cr",
        "EBITDA Margin": r"EBITDA Margin\s+([\d\.]+)\s*%",
        "PAT Margin": r"PAT Margin\s+([\d\.]+)\s*%",
        "Free Cash Flow": r"Free Cash Flow.*?([\d\.\-]+)\s*Cr",
        "Cash & Cash Equivalents": r"Cash & Cash Equivalents.*?([\d\.]+)\s*Cr",
    }

    results = {}
    for metric_name, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            results[metric_name] = match.group(1).strip()

    return results

# Function to insert metrics into the database
def insert_metrics_to_db(metrics, source_pdf):
    conn = connect_postgres()
    cur = conn.cursor()
    for metric_name, metric_value in metrics.items():
        cur.execute(
            """
            INSERT INTO metric_table (metric_name, metric_value, source_pdf, extraction_date)
            VALUES (%s, %s, %s, %s)
            """,
            (metric_name, metric_value, source_pdf, datetime.now())
        )
    conn.commit()
    cur.close()
    conn.close()
    print(f"Inserted {len(metrics)} metrics from {source_pdf}.")

# Execution Block
if __name__ == "__main__":
    # Load extracted text file (replace with your actual file path)
    with open("extracted_text/Q1FY24.txt", "r", encoding="utf-8") as f:
        text_content = f.read()
    
    # Extract metrics
    metrics = extract_metrics(text_content)
    print("Extracted Metrics:", metrics)

    # Insert into PostgreSQL
    insert_metrics_to_db(metrics, "Q1FY24.pdf")


Extracted Metrics: {'Revenue from Operations': '19.8', 'EBITDA': '0.02', 'PAT': '0.7', 'EBITDA Margin': '0.1', 'PAT Margin': '3.5'}
Inserted 5 metrics from Q1FY24.pdf.
